In [1]:
import pyprind
import os
import numpy as np
import pandas as pd
import nltk
import re
import string

In [2]:
# ls

In [3]:
# basepath = 'C:/Users/rpatil/Desktop/rpatil/Raunak/Documents/Python Scripts/DS 2 Mid Term 1\Data'
# labels = {'pos': 1, 'neg': 0}
# pbar = pyprind.ProgBar(50000)
# df = pd.DataFrame()
# for s in ('test', 'train'):
#     for l in ('pos', 'neg'):
#         path = os.path.join(basepath, s, l)
#         for file in os.listdir(path):
#             with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
#                 txt = infile.read()
#             df = df.append([[txt, labels[l]]], ignore_index=True)
#             pbar.update()
# df.columns = ['review', 'sentiment']

In [4]:
data=pd.read_csv('movie_data.csv')
print(data.head())
print(data.shape)

                                              review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0
3  hi for all the people who have seen this wonde...          1
4  I recently bought the DVD, forgetting just how...          0
(50000, 2)


In [5]:
total = data['sentiment'].sum()
total

25000

In [6]:
#2 Cleaning the Text

#2.1 Removing Punctuation
#2.2 Removing Stop words
#2.3 Stemming

In [7]:
punc_set = string.punctuation
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_text(text):
    text="".join([x.lower() for x in text if x not in punc_set])
    tokens=re.split("\W+",text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

data['clean_text']= data['review'].apply(lambda z:clean_text(z))

In [8]:
data['clean_text'][0]

['1974',
 'teenag',
 'martha',
 'moxley',
 'maggi',
 'grace',
 'move',
 'highclass',
 'area',
 'bell',
 'greenwich',
 'connecticut',
 'mischief',
 'night',
 'eve',
 'halloween',
 'murder',
 'backyard',
 'hous',
 'murder',
 'remain',
 'unsolv',
 'twentytwo',
 'year',
 'later',
 'writer',
 'mark',
 'fuhrman',
 'christoph',
 'meloni',
 'former',
 'la',
 'detect',
 'fallen',
 'disgrac',
 'perjuri',
 'oj',
 'simpson',
 'trial',
 'move',
 'idaho',
 'decid',
 'investig',
 'case',
 'partner',
 'stephen',
 'week',
 'andrew',
 'mitchel',
 'purpos',
 'write',
 'book',
 'local',
 'squirm',
 'welcom',
 'support',
 'retir',
 'detect',
 'steve',
 'carrol',
 'robert',
 'forster',
 'charg',
 'investig',
 '70',
 'discov',
 'crimin',
 'net',
 'power',
 'money',
 'cover',
 'murderbr',
 'br',
 'murder',
 'greenwich',
 'good',
 'tv',
 'movi',
 'true',
 'stori',
 'murder',
 'fifteen',
 'year',
 'old',
 'girl',
 'commit',
 'wealthi',
 'teenag',
 'whose',
 'mother',
 'kennedi',
 'power',
 'rich',
 'famili',
 '

In [9]:
#PART 3 Creating 3 types of word conversion to matrices
#3.1 N-grams
#3.2 TF-IDF
#We will apply all types to all 3 models to see best accuracy.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
x_tfidf = tfidf_vect.fit_transform(data["clean_text"])

print (x_tfidf.shape)
print (tfidf_vect.get_feature_names())

(50000, 49577)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score


rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)

cross_val_score (rf , x_tfidf, data["sentiment"], cv = k_fold, scoring='accuracy',n_jobs=-1)

array([0.5053, 0.4957, 0.5031, 0.5017, 0.505 ])

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score


x_train, x_test, y_train, y_test = train_test_split(x_tfidf,data["sentiment"],test_size=0.5)

In [13]:
rf = RandomForestClassifier(n_estimators=50,max_depth=20)
rf_model = rf.fit(x_train,y_train)

In [14]:
y_pred = rf_model.predict(x_test)

precision, recall, fscore, support= score(y_test,y_pred,pos_label=0,average="binary")

In [15]:
print("precision:{}, recall:{}, accuracy:{}".format(precision,recall,(y_pred==y_test).sum()/len(y_pred)))

precision:1.0, recall:0.0008725311334972634, accuracy:0.49616


In [16]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)
average_precision

0.495938212813638

In [28]:
X_train = data.loc[:25000, 'review'].values
y_train = data.loc[:25000, 'sentiment'].values
X_test = data.loc[25000:, 'review'].values
y_test = data.loc[25000:, 'sentiment'].values

In [25]:
# from sklearn.feature_extraction.text import CountVectorizer

# cv = CountVectorizer(analyzer=clean_text)
# x_count = cv.fit_transform(data['clean_text'])

In [29]:
log_cv = KFold(n_splits=10,random_state=seed,shuffle=True)
log_avg = 0
i=1
print(0)

from sklearn.linear_model import LogisticRegression

log = LogisticRegression(C=10 ,  penalty = 'l2' ,random_state=0, solver='lbfgs', multi_class='multinomial')
log.fit(X_train, y_train)
log_prediction = log.predict(x_valid)
log_r2 = log.score(x_test,y_test)
log_avg = log_avg + log_r2
print('Accuracy score for Log-Regression = ',log_r2)
i+=1
print('Average Accuracy score for Logistic Regression = ',log_avg/10)

IndentationError: unexpected indent (<ipython-input-29-abecf239a7e0>, line 9)